<a href="https://colab.research.google.com/github/Ruheena-S/Hierarchical-classification-Loss-Functions-in-Image-Classification/blob/main/BEP_ResNet50_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Tue Mar 14 01:26:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    34W /  70W |  14783MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-------------

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import sys
import torch.nn.functional as F
from torch_optimizer import Ranger
from torchvision import datasets
import torch.optim as optim
from torchvision import transforms,models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
#Tree Hierarchy of the labels

Tree = {"entity" : ["nature", "manmade"] , "nature":["flora", "bigAnimals", "smallAnimals", "aquaticAnimals", "scenes"], "manmade" : ["householdElectrical", "householdFurniture", "foodContainers", "vehicles1","vehicles2","largeManmadeOutdoorThings"],
        "flora": ["fruitsAndVegetables","flowers","trees"],"bigAnimals": ["largeCarnivores", "largeOmniAndHerbivores", "people", "mediumSizedMammals"],"smallAnimals":["smallMammals", "reptiles", "insects", "nonInsectInvertebrates"], "aquaticAnimals":["aquaticMammals", "fish"],"scenes":["cloud","forest","mountain","plain","sea"],
        "householdElectrical":["clock", "keyboard", "lamp", "telephone", "television"], "householdFurniture":["bed", "chair", "couch", "table", "wardrobe"],"foodContainers":["bottle", "bowl", "can", "cup", "plate"], "vehicles1":["bicycle", "bus", "motorcycle", "pickup_truck", "train"],"vehicles2":["lawn_mower", "rocket", "streetcar", "tank", "tractor"],"largeManmadeOutdoorThings":["bridge", "castle", "house", "road", "skyscraper"],
        "flowers":["orchid", "poppy", "rose", "sunflower", "tulip"],"fruitsAndVegetables":["apple", "mushroom", "orange", "pear", "sweet_pepper"],"trees":["maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree"],"largeCarnivores":["bear", "leopard", "lion", "tiger", "wolf"],"largeOmniAndHerbivores":["camel", "cattle", "chimpanzee", "elephant", "kangaroo"],"people":["baby", "boy", "girl", "man", "woman"],
        "mediumSizedMammals":["fox", "porcupine", "possum", "raccoon", "skunk"],"smallMammals":["hamster", "mouse", "rabbit", "shrew", "squirrel"],"reptiles":["crocodile", "dinosaur", "lizard", "snake", "turtle"],"insects":["bee", "beetle", "butterfly", "caterpillar", "cockroach"],"nonInsectInvertebrates":["crab", "lobster", "snail", "spider", "worm"],"aquaticMammals":["beaver", "dolphin", "otter", "seal", "whale"],
        "fish":["aquarium_fish", "flatfish", "ray", "shark", "trout"]}

labels = ["apple","aquarium_fish","baby","bear","beaver","bed","bee","beetle","bicycle","bottle","bowl","boy","bridge","bus","butterfly","camel","can","castle","caterpillar","cattle","chair","chimpanzee","clock","cloud","cockroach","couch","crab","crocodile","cup","dinosaur","dolphin","elephant","flatfish","forest","fox","girl","hamster","house","kangaroo","keyboard","lamp","lawn_mower","leopard","lion","lizard","lobster","man","maple_tree","motorcycle","mountain","mouse","mushroom","oak_tree","orange","orchid","otter","palm_tree","pear","pickup_truck","pine_tree","plain","plate","poppy","porcupine","possum","rabbit","raccoon","ray","road","rocket","rose","sea","seal","shark","shrew","skunk","skyscraper","snail","snake","spider","squirrel","streetcar","sunflower","sweet_pepper","table","tank","telephone","television","tiger","tractor","train","trout","tulip","turtle","wardrobe","whale","willow_tree","wolf","woman","worm", "flowers","fruitsAndVegetables", "trees", "largeCarnivores", "largeOmniAndHerbivores", "people", "mediumSizedMammals", "smallMammals", "reptiles", "insects", "nonInsectInvertebrates", "aquaticMammals", "fish", "flora", "bigAnimals", "smallAnimals","aquaticAnimals", "scenes", "householdElectrical", "householdFurniture", "foodContainers", "vehicles1", "vehicles2", "largeManmadeOutdoorThings", "nature" ,"manmade", "entity"]


In [ ]:
#Level-wise representation of labels

level = [None]*5

level[0] = ["entity"]
level[1] = ["nature", "manmade"]
level[2] = ["flora", "bigAnimals", "smallAnimals", "aquaticAnimals", "scenes", "householdElectrical", "householdFurniture","foodContainers", "vehicles1", "vehicles2","largeManmadeOutdoorThings"]
level[3] = ["flowers", "fruitsAndVegetables", "trees", "largeCarnivores", "largeOmniAndHerbivores", "people", "mediumSizedMammals", "smallMammals", "reptiles", "insects", "nonInsectInvertebrates", "aquaticMammals", "fish", "cloud", "forest", "mountain", "plain", "sea", "clock", "keyboard", "lamp", "telephone", "television", "bed", "chair", "couch", "table", "wardrobe", "bottle", "bowl", "can", "cup", "plate", "bicycle", "bus", "motorcycle", "pickup_truck", "train", "lawn_mower", "rocket", "streetcar", "tank", "tractor", "bridge", "castle", "house", "road", "skyscraper"]
level[4] = ["orchid", "poppy", "rose", "sunflower", "tulip", "apple", "mushroom", "orange", "pear", "sweet_pepper", "maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree", "bear", "leopard", "lion", "tiger", "wolf", "camel", "cattle", "chimpanzee", "elephant", "kangaroo", "baby", "boy", "girl", "man", "woman", "fox", "porcupine", "possum", "raccoon", "skunk", "hamster", "mouse", "rabbit", "shrew", "squirrel", "crocodile", "dinosaur", "lizard", "snake", "turtle", "bee", "beetle", "butterfly", "caterpillar", "cockroach", "crab", "lobster", "snail", "spider", "worm", "beaver", "dolphin", "otter", "seal", "whale", "aquarium_fish", "flatfish", "ray", "shark", "trout"]


nodesAtLevel = []*5
nodesAtLevel.append(1)
nodesAtLevel.append(2)
nodesAtLevel.append(11)
nodesAtLevel.append(48)
nodesAtLevel.append(65)

logNodesAtLevel = []*5
logNodesAtLevel.append(1)
logNodesAtLevel.append(1)
logNodesAtLevel.append(4)
logNodesAtLevel.append(6)
logNodesAtLevel.append(7)


In [ ]:
# Input    - class label of the CIFAR100 dataset
# Function - To find the level of the given class label
# output   - Returns an integer which represents the level of the class label and returns -1 if it doesn't belong to any level
def get_level(node):
  if (node in level[0]):
    return 0
  elif (node in level[1]):
    return 1
  elif (node in level[2]):
    return 2
  elif (node in level[3]):
    return 3
  elif (node in level[4]):
    return 4
  else:
    return -1

# Input    - class label of the CIFAR100 dataset
# Function - To find all the children of the given class label using the Tree hierarchy
# output   - Returns an list of the labels which are children and returns None if the given label is a leaf
def get_children(node):
  if node in Tree.keys():
    return Tree[node]    
  else:
    return None

# Input    - class label of the CIFAR100 dataset
# Function - To find the parent of the given class label using the level and children function.
# output   - Returns the parent of the class label
def get_parent(node):
  l = get_level(node)
  if l == 0: 
    return node
  for i in level[l-1]:
    if(get_children(i) is not None):
      if node in get_children(i):
        return i
  return None

# Input    - class label of the CIFAR100 dataset and the level of the label
# Function - finds ancestors of the the given label. 
# output   - Returns the label itself if given level is greater than the label's level else returns the ancestor at level l
def get_ancestor(node, l):
  h = get_level(node)
  if l >= h:
    return node
  y = node
  for i in range(h-l):
    y = get_parent(y)
  return y

# Input    - class label of the CIFAR100 dataset
# Function - To find the descendants of the given class label including the given class label
# output   - Returns an list of class labels which are descendants else empty list if the class label is a leaf.
def get_descendants(node):
  c = get_children(node)
  d = []
  if c is not None:
    for i in c:
      d.extend(get_descendants(i)) 
    return d
  else :
    return [node] 


# Input    - two class labels of the CIFAR100 dataset
# Function - Finds the distance between the given two nodes
# output   - Returns an integer which represents the distance between class labels
def tree_loss(node1,node2):
  l1 = get_level(node1)
  l2 = get_level(node2)
  l = min(l1,l2)
  while l >= 0:
    if get_ancestor(node1,l) == get_ancestor(node2,l):
      break
    else :
      l = l-1
  return l1 + l2 - 2*l


In [ ]:
def tLoss(output,target,tau_values=[0.5,0.5,0.5,0.5,0.5]):
  batch_size = target.size(0)
  num_classes = output.size(1)
  h = 4
  val = None
  pred = None
  loss = 0
  global labels
  global level
  l1 = logNodesAtLevel[0]
  l2 = logNodesAtLevel[1] + l1
  l3 = logNodesAtLevel[2] + l2
  l4 = logNodesAtLevel[3] + l3
  l5 = logNodesAtLevel[4] + l4

  # Loop for each example in the batch
  for i in range(batch_size):
    # t is the actual target of the ith example in the batch
    t = target[i]
    # Go searching bottom-up in the hierarchy tree
    h = 4
    while h >= 0:
      
      # At each level in the bottom-up traversal, we take the minimum of the outputs of nodes 
      # Based on the height consider the range in the totat set of output nodes of the model
      # Get the min value of the absolute values
      # If the min val isgreater than the threshold at that level the we break and get class index at level from 
      # the signs of those outputs are defined in btoi function
      values = output.clone().detach()
      if h == 4:        
        values = values[i,l4:l5]        
        val,_ = torch.min(torch.abs(values),0)   
        if val >= tau_values[4]:
          pred = b2i(values,h)
          pred_label = labels[pred]
          break

      if h == 3:
        values = values[i,l3:l4]
        val,_ = torch.min(torch.abs(values),0)        
        if val >= tau_values[3]:
          pred = b2i(values,h)
          pred_label = level[h][pred]
          break

      if h == 2:
        values = values[i,l2:l3]
        val,_ = torch.min(torch.abs(values),0)        
        if val >= tau_values[2]:
          pred = b2i(values,h)
          pred_label = level[h][pred]
          break

      if h == 1:
        values = values[i,l1:l2]
        val,_ = torch.min(torch.abs(values),0)        
        if val >= tau_values[1]:
          pred = b2i(values,h)
          pred_label = level[h][pred]
          break

      if h == 0:
        values = values[i,0:l1]
        val,_ = torch.min(torch.abs(values),0)        
        if val >= tau_values[0]:
          pred = b2i(values,h)
          pred_label = level[h][pred]
          break 
          
      h = h - 1

    # tree_loss() function gives the tree distance between the two nodes
    # Total loss for the batch is calculated
    loss = loss + tree_loss(labels[t],pred_label)

  loss = loss/batch_size

  # Average loss for the batch is returned
  return loss

In [ ]:
# function to convert integer to binary
def itob(target,bits):
  encoding=-np.ones(bits)
  ''' -1 in place of 0 in binary representation of a number'''
  j = bits-1
  while(target!=0):
    if (target%2)==1 :
      encoding[j] = 1
    target = target//2
    j = j-1
  return encoding

In [ ]:
#function to convert a batch of binary numbers into integer
def btoi(binary,h):
  # this is to convert a -1 and 1 to 0 and 1 respectively
  batch_size = binary.size(0)
  binary = -torch.sign(binary)
  bits = logNodesAtLevel[h]
  binary = (binary+1)/2
  #binary = torch.ceil(binary)

  j = bits-1
  nodes = [1,2,11,48,100]
  #j = bits-1 
  decoded_target = torch.zeros(batch_size).to(device)
  while j>=0:
    decoded_target = decoded_target + binary[:,j] * (2**(bits-j-1))
    j = j-1
  
  return decoded_target

#function to convert a single binary number into integer
def b2i(binary,h):
  batch_size = binary.size(0)
  binary = -torch.sign(binary)
  bits = logNodesAtLevel[h]
  binary = (binary+1)/2
  #binary = torch.ceil(binary)
  #print(binary.shape)
  j = bits-1
  nodes = [1,2,11,48,100]

  max_label = nodes[h]
  decoded_target = 0
  while j>=0:
    decoded_target = decoded_target + binary[j] * (2**(bits-j-1))
    j = j-1
  if decoded_target >= max_label:
    decoded_target = decoded_target - 2**(bits-1)
  return decoded_target.type(torch.LongTensor)

In [ ]:
def target_translate(target):
    #print(target)
    global labels
    global level
    global logNodesAtLevel
    
    translated_target = []  # target is translated into 19 bits (1+1+4+6+7)
    # 5 is the height of the tree
    target_label = labels[target]
    for i in range(4):
      ancestor = get_ancestor(target_label,i)
      #print(ancestor)
      if ancestor==target_label and (ancestor not in level[i]):
        print(ancestor)
        translated_target.extend(itob(nodesAtLevel[i]+1,logNodesAtLevel[i]))
      else:
        translated_target.extend(itob(level[i].index(ancestor),logNodesAtLevel[i]))
        
    translated_target.extend(itob(target,logNodesAtLevel[4]))
    translated_target = torch.FloatTensor(translated_target)

    return target, translated_target

In [ ]:
# Define transforms for the dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


dataset = datasets.CIFAR100(root='data/', train=True, download=True, transform=transform_train,target_transform=target_translate)
test_dataset = datasets.CIFAR100(root='data/', train=False, download=True, transform=transform_test,target_transform=target_translate)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=256

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_ds, batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
num_classes = 19
num_epochs = 50

In [ ]:
class BEP_Loss(torch.nn.Module):
  def __init__(self):
        super(BEP_Loss,self).__init__()
        
  def forward(self,y_p,y_t):
    
    batch_size = y_p.size(0)  # shape of y_p is batch_size * 19

    z = y_p*y_t              # shape of z is batch_size * 19

    start = 0   
    loss = 0 
    zeros = torch.zeros(batch_size).to(device) #shape of zeros is batch_size
    
    for i in range(5):
      t1 = z[:,start:start+logNodesAtLevel[i]]
      w,_ = torch.max(t1,1)
      w1 = torch.max(w+1,zeros)
      loss = loss + torch.sum(w1)
      start = start+logNodesAtLevel[i]

    loss = loss/batch_size
    return loss

In [ ]:
# Define the ResNet50 model and freeze layers

model = models.resnet50(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size = (3,3), padding = (1, 1), bias = False)
model.maxpool = nn.Identity()

for param in model.parameters():
    param.requires_grad = True
# for param in model.layer4.parameters():
#     param.requires_grad = True

model.fc = nn.Linear(2048, 19)

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [ ]:
# Define loss function and optimizer

criterion = BEP_Loss().to(device)
accuracy = Accuracy(task="multiclass",num_classes=num_classes).to(device)
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = Ranger(model.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)

In [ ]:
def get_accuracy(output, target):

  batch_size = target.size(0)
  
  #checking the values of logm labels
  pred=btoi(output[:,12:19],4)

  correct = pred==target

  return correct.float().sum()/batch_size

In [ ]:
#Training

total_train_step = len(train_loader)
total_val_step=len(valid_loader)

BEST_VAL_METRIC = 0
BEST_MODEL = None


for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0.0
    model.train()

    for i, (images, target) in enumerate(train_loader, 1):
        
        y_true = target[0]
        y_trans = target[1]

        # Move tensors to the configured device
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, y_trans)

        train_loss += loss
        train_acc += get_accuracy(outputs,y_true)
        #train_acc += accuracy(btoi(outputs[:,12:19],4), y_true)
        
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
      
    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')
    
    # Validation
    
    model.eval() 
    
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        Tree_Loss_Value = 0
        for i, (images, target) in enumerate(valid_loader, 1):

            y_true = target[0]
            y_trans = target[1]

            # Move tensors to the configured device
            images = images.to(device)
            y_true = y_true.to(device)
            y_trans = y_trans.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, y_trans)
            Tree_Loss_Value += tLoss(outputs,y_true)
            val_acc += get_accuracy(outputs,y_true)
            #val_acc += accuracy(btoi(outputs[:,12:19],4), y_true)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = model.state_dict() 
        torch.save(model, "/content/drive/MyDrive/MTP_Phase2/saved_models/resnet50_BEPCascade_cifar100_RangerOPT.pth")
        #torch.save(model, "/content/drive/MyDrive/Ruheena/MTP/saved_models/resnet50_BEPCascade_cifar100_AdamOPT.pth")

    print(f'Accuracy of the network on validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}, Tree loss: {(Tree_Loss_Value/total_val_step):.4f}') 


/usr/local/lib/python3.9/dist-packages/pytorch_ranger/ranger.py:172: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Epoch [1/50] - Loss: 4.1160, Accuracy: 0.0124
Accuracy of the network on validation images: 0.0166, loss: 3.2675, Tree loss: 2.8362
Epoch [2/50] - Loss: 3.1494, Accuracy: 0.0303
Accuracy of the network on validation images: 0.0687, loss: 2.9595, Tree loss: 2.5618
Epoch [3/50] - Loss: 2.7598, Accuracy: 0.0886
Accuracy of the network on validation images: 0.1766, loss: 2.5056, Tree loss: 2.0807
Epoch [4/50] - Loss: 2.3960, Accuracy: 0.1911
Accuracy of the network on validation images: 0.2836, loss: 2.2058, Tree loss: 1.8359
Epoch [5/50] - Loss: 2.1214, Accuracy: 0.2684
Accuracy of the network on validation images: 0.3624, loss: 1.9663, Tree loss: 1.6271
Epoch [6/50] - Loss: 1.9102, Accuracy: 0.3510
Accuracy of the network on validation images: 0.4445, loss: 1.8005, Tree loss: 1.4874
Epoch [7/50] - Loss: 1.7233, Accuracy: 0.4191
Accuracy of the network on validation images: 0.4740, loss: 1.6807, Tree loss: 1.3847
Epoch [8/50] - Loss: 1.5989, Accuracy: 0.4709
Accuracy of the network on val

In [ ]:

#Testing
model.load_state_dict(BEST_MODEL)

total_test_step=len(test_loader)

with torch.no_grad():
    test_acc=0
    test_loss=0
    Tree_Loss_Value=0

    for i, (images, target) in enumerate(test_loader, 1):
        
        y_true = target[0]
        y_trans = target[1]
        
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)

        # Forward pass
        outputs = model(images)
        
        # Loss
        test_loss += criterion(outputs,y_trans)
        Tree_Loss_Value += tLoss(outputs,y_true)
        test_acc += get_accuracy(outputs, y_true)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}, Tree loss: {(Tree_Loss_Value/total_test_step):.4f}')

Accuracy of the network on test images: 0.7397, loss: 1.1963, Tree loss: 0.8416
